In [1]:
%pip install -U openai-whisper

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 792 kB 1.3 MB/s eta 0:00:01
  distutils: /private/var/folders/sq/pjwy2d1s15q5w6qp6w5l6bp40000gn/T/pip-build-env-dlv2ms32/normal/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  distutils: /private/var/folders/sq/pjwy2d1s15q5w6qp6w5l6bp40000gn/T/pip-build-env-dlv2ms32/normal/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  user = False
  home = None
  root = None
  prefix = '/private/var/folders/sq/pjwy2d1s15q5w6qp6w5l6bp40000gn/T/pip-build-env-dlv2ms32/normal'
  distutils: /private/var/folders/sq/pjwy2d1s15q5w6qp6w5l6bp40000gn/T/pip-build-env-dlv2ms32/overlay/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  distutils: /private/var/folders/sq/pjwy2d1s15q5w6qp6w5l6bp40000gn/T/pip-build-env-dlv2ms32/overlay/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  user = 

In [2]:
import whisper
model = whisper.load_model('base')

/Users/minseokbae/Library/Python/3.9/lib/python/site-packages/whisper/timing.py:58: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def backtrace(trace: np.ndarray):
100%|███████████████████████████████████████| 139M/139M [00:31<00:00, 4.66MiB/s]


In [4]:
%pip install "ray[serve]" 
%pip install transformers
%pip install requests
%pip install torch

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip in

In [21]:
%pip install soundfile

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.1 MB 4.0 MB/s eta 0:00:01
     |████████████████████████████████| 174 kB 4.1 MB/s eta 0:00:01
  Using cached pycparser-2.21-py2.py3-none-any.whl (118 kB)
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [26]:
import ray
from ray import serve
import requests
from starlette.requests import Request
import soundfile as sf
import io

@serve.deployment(num_replicas=2, ray_actor_options={"num_cpus": 0.2, "num_gpus": 0})
class SpeechToText:
    def __init__(self):
        self.model = whisper.load_model('base')
    
    def transcribe(self, audio_bytes):
        # Convert audio bytes to a NumPy array
        audio, _ = sf.read(io.BytesIO(audio_bytes))
        
        # Now transcribe using the whisper model
        model_output = self.model.transcribe(audio, verbose=True)  
        return model_output['text']
    
    async def __call__(self, request: Request) -> str:
        if request.method == "POST":
            audio_bytes = await request.body()
            transcription = self.transcribe(audio_bytes)
            return transcription
        else:
            return "Only POST requests are supported."

In [10]:
ray.shutdown()
ray.init()

2023-09-14 21:49:14,243	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.9.6
Ray version:,2.6.3
Dashboard:,http://127.0.0.1:8265


In [11]:
serve.start()

(HTTPProxyActor pid=99982) INFO:     Started server process [99982]


In [27]:
# Deploy the transcription app
SpeechToText.deploy()

(ServeController pid=99979) INFO 2023-09-14 22:13:29,673 controller 99979 deployment_state.py:1308 - Deploying new version of deployment SpeechToText.
(ServeController pid=99979) INFO 2023-09-14 22:13:29,706 controller 99979 deployment_state.py:1459 - Stopping 1 replicas of deployment 'SpeechToText' with outdated versions.
(ServeController pid=99979) INFO 2023-09-14 22:13:31,881 controller 99979 deployment_state.py:1571 - Adding 1 replica to deployment SpeechToText.
(ServeReplica:SpeechToText pid=3563) /Users/minseokbae/Library/Python/3.9/lib/python/site-packages/whisper/timing.py:58: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
(ServeReplica:SpeechToText pid=3563)   def bac

In [28]:
audio_file = './harvard.wav'

response = requests.post("http://127.0.0.1:8000/SpeechToText", data=open(audio_file, 'rb').read())

print(response)

(ServeReplica:SpeechToText pid=3563) /Users/minseokbae/Library/Python/3.9/lib/python/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
(ServeReplica:SpeechToText pid=3563)   warnings.warn("FP16 is not supported on CPU; using FP32 instead")
(ServeController pid=99979) WARNING 2023-09-14 22:19:43,559 controller 99979 deployment_state.py:662 - Didn't receive health check response for replica SpeechToText#tudMgI after 30.0s, marking it unhealthy.
(ServeController pid=99979) WARNING 2023-09-14 22:20:13,616 controller 99979 deployment_state.py:662 - Didn't receive health check response for replica SpeechToText#tudMgI after 30.0s, marking it unhealthy.


<Response [500]>


(ServeController pid=99979) WARNING 2023-09-14 22:20:43,695 controller 99979 deployment_state.py:662 - Didn't receive health check response for replica SpeechToText#tudMgI after 30.0s, marking it unhealthy.
(ServeController pid=99979) WARNING 2023-09-14 22:20:43,696 controller 99979 deployment_state.py:727 - Replica SpeechToText#tudMgI failed the health check 3 times in a row, marking it unhealthy.
(ServeController pid=99979) WARNING 2023-09-14 22:20:43,697 controller 99979 deployment_state.py:1812 - Replica SpeechToText#tudMgI of deployment SpeechToText failed health check, stopping it.
(ServeController pid=99979) INFO 2023-09-14 22:20:43,716 controller 99979 deployment_state.py:911 - Replica SpeechToText#tudMgI did not shut down after grace period, force-killing it. 


(HTTPProxyActor pid=99982) ERROR 2023-09-14 22:20:43,740 http_proxy 127.0.0.1 tdVbmNKNGd /SpeechToText http_proxy.py:819 - The actor died unexpectedly before finishing this task.
(HTTPProxyActor pid=99982) 	class_name: ServeReplica:SpeechToText
(HTTPProxyActor pid=99982) 	actor_id: c994c4020bd1a470e3e90e8801000000
(HTTPProxyActor pid=99982) 	pid: 3563
(HTTPProxyActor pid=99982) 	name: SERVE_REPLICA::SpeechToText#tudMgI
(HTTPProxyActor pid=99982) 	namespace: serve
(HTTPProxyActor pid=99982) 	ip: 127.0.0.1
(HTTPProxyActor pid=99982) The actor is dead because it was killed by `ray.kill`.
(HTTPProxyActor pid=99982) Traceback (most recent call last):
(HTTPProxyActor pid=99982)   File "/Users/minseokbae/Library/Python/3.9/lib/python/site-packages/ray/serve/_private/http_proxy.py", line 802, in send_request_to_replica_streaming
(HTTPProxyActor pid=99982)     status_code = await self._consume_and_send_asgi_message_generator(
(HTTPProxyActor pid=99982)   File "/Users/minseokbae/Library/Python/3